Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Architecture for Grounding with Google Search
 * [Grounding with Google search](https://cloud.google.com/static/vertex-ai/generative-ai/docs/images/gemini/overview-grounding.png)

* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [4]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # Mount to the google drive to access the .ipynb files in the repository.
    # from google.colab import drive
    # drive.mount('/content/drive')

## Set the environment on GCP Project


In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [14]:
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Prompt sent to LLM

In [7]:
prompt ="민간 부문의 양자 정보 통신 기술 개발 지원에 대해서 설명해주세요."

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

In [8]:
response = model.generate_content(prompt)
display(Markdown(response.text))

## 민간 부문의 양자 정보 통신 기술 개발 지원: 잠재력과 과제

양자 정보 통신 기술은 기존 통신 시스템의 한계를 뛰어넘는 혁신적인 기술로, 보안성, 속도, 용량 측면에서 비약적인 발전을 가능하게 합니다. 이러한 잠재력을 실현하기 위해 민간 부문의 활발한 참여와 지원이 필수적입니다.

**1. 민간 부문 지원의 중요성:**

* **혁신 주도:** 민간 기업들은 시장 수요와 기술 발전에 민감하게 반응하며 혁신적인 양자 정보 통신 기술 개발을 선도할 수 있습니다.
* **투자 확대:** 민간 투자는 연구 개발, 인프라 구축, 상용화 등 양자 정보 통신 기술 발전에 필요한 자원을 확보하는 데 중요한 역할을 합니다.
* **시장 확대:** 민간 기업들은 개발된 기술을 시장에 출시하고 새로운 서비스를 창출하여 양자 정보 통신 시장을 확대하는 데 기여합니다.

**2. 지원 방식:**

* **R&D 투자:** 정부는 양자 정보 통신 기술 개발을 위한 연구 개발 지원 프로그램을 마련하고, 민간 기업의 참여를 유도해야 합니다.
* **인프라 구축:** 양자 통신 네트워크 구축에 필요한 기반 시설 지원 및 표준화 작업을 추진하여 민간 기업의 참여를 용이하게 해야 합니다.
* **규제 완화:** 양자 정보 통신 기술의 상용화를 위한 규제를 완화하고 새로운 서비스 출시를 위한 환경을 조성해야 합니다.
* **인력 양성:** 양자 정보 통신 분야의 전문 인력 양성을 위한 교육 프로그램을 지원하고, 민간 기업의 인력 확보를 지원해야 합니다.
* **협력 네트워크 구축:** 정부, 학계, 민간 기업 간 협력 네트워크를 구축하여 정보 공유, 기술 교류, 공동 연구를 활성화해야 합니다.

**3. 과제:**

* **고비용:** 양자 정보 통신 기술 개발 및 상용화에는 상당한 비용이 소요되어 민간 기업의 투자 유치가 어려울 수 있습니다.
* **기술적 난제:** 양자 정보 통신 기술은 아직 초기 단계에 있으며 해결해야 할 기술적 난제가 많습니다.
* **시장 성숙:** 양자 정보 통신 시장은 아직 성숙되지 않아 초기 투자 대비 수익 창출이 불확실합니다.

**4. 결론:**

민간 부문의 활발한 참여는 양자 정보 통신 기술 발전의 핵심입니다. 정부는 민간 기업의 투자를 유치하고 기술 개발을 지원하는 다양한 정책을 추진해야 합니다. 뿐만 아니라, 기술적 난제 해결, 시장 성숙, 인력 양성 등 다각적인 노력을 통해 양자 정보 통신 시대를 앞당길 수 있을 것입니다.


##Configuration Vertex AI Search

In [19]:
DATA_STORE_PROJECT_ID = PROJECT_ID
DATA_STORE_REGION = "global"

# Note : Only Digital parser datastore is possible to us grounding service as of Aug 2024
DATA_STORE_ID = "hackathon-ds_1709838247983"


## Execute request to LLM with grounding servcies supported by Vertex AI search.

In [20]:

# datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"

data_store_path = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"


tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=data_store_path))
)

response = model.generate_content(prompt, tools=[tool])

display(Markdown(response.text))

2023년 10월 31일부터 정보통신 진흥 및 융합 활성화 등에 관한 특별법(제19240호) 제27조의3이 삭제되어 민간 부문의 양자 정보 통신 기술 개발 지원에 대한 법적 근거가 사라졌습니다. 따라서 현재는 민간 부문의 양자 정보 통신 기술 개발 지원에 대한 명확한 법적 규정이 없습니다. 
